# Preprocessing

In [1]:
import pandas as pd
import numpy as np
import re
import spacy
import spacy.symbols

c:\Users\danie\anaconda3\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


### Getting data

In [14]:
comments_df = pd.read_csv('../data/full_df.csv')
#"C:\Users\danie\Study\MACSS\macs30200\scraper\top_100_post_comments_user_time_text.txt"
print(comments_df.shape)
comments_df

(4861, 7)


,id,parent_id,username,time_created,flair,body,subreddit
0,14m8mf4,NaN,Tungsten_,2023-06-29 10:54:44,NaN,[Megathread] Supreme Court Ruling on Affirmati...,t5_2rfyw
1,jq5du0z,t3_14m8mf4,Tungsten_,2023-06-30 11:33:11,NaN,Thanks to everyone who engaged in insightful a...,t5_2rfyw
2,jq0dgzx,t3_14m8mf4,ProudBlackMatt,2023-06-29 11:16:15,Chinese-American,I would prefer using a process that takes into...,t5_2rfyw
3,jq0cg7k,t3_14m8mf4,TomatoCanned,2023-06-29 11:09:47,NaN,"u/Tungsten_, Thanks for creating a section jus...",t5_2rfyw
4,jq0f52k,t3_14m8mf4,bad-fengshui,2023-06-29 11:26:41,NaN,As with anything related to Asians in politics...,t5_2rfyw
...,...,...,...,...,...,...,...
4856,osjkh6,NaN,yellowmix,2021-07-27 09:32:14,NaN,This Is the End of Affirmative Action. What ar...,t5_2qhgd
4857,iw0q5sn,t3_yr5o90,yellowmix,2022-11-12 01:09:36,NaN,What do you mean? That nepotism comes from whi...,t5_2qhgd
4858,uyzxgz,NaN,Kamala_Metamorph,2022-05-27 14:52:16,​,How to have a conversation with an open-minded...,t5_38jid
4859,l20ftbs,t3_1cfw1ru,Extension_River_9901,2024-04-30 22:56:39,New user,Democrats that want to expand education .Fun...,t5_3amv4


#### Drop rows with deleted text body

In [15]:
deleted_rows = comments_df[comments_df['body'].isin(['[deleted]','[removed]'])]
print(f'Num of deleted rows: {deleted_rows.shape[0]}')

# drop rows
comments_df.drop(deleted_rows.index, inplace=True)
comments_df.reset_index(drop=True, inplace=True)
comments_df

Num of deleted rows: 307


,id,parent_id,username,time_created,flair,body,subreddit
0,14m8mf4,NaN,Tungsten_,2023-06-29 10:54:44,NaN,[Megathread] Supreme Court Ruling on Affirmati...,t5_2rfyw
1,jq5du0z,t3_14m8mf4,Tungsten_,2023-06-30 11:33:11,NaN,Thanks to everyone who engaged in insightful a...,t5_2rfyw
2,jq0dgzx,t3_14m8mf4,ProudBlackMatt,2023-06-29 11:16:15,Chinese-American,I would prefer using a process that takes into...,t5_2rfyw
3,jq0cg7k,t3_14m8mf4,TomatoCanned,2023-06-29 11:09:47,NaN,"u/Tungsten_, Thanks for creating a section jus...",t5_2rfyw
4,jq0f52k,t3_14m8mf4,bad-fengshui,2023-06-29 11:26:41,NaN,As with anything related to Asians in politics...,t5_2rfyw
...,...,...,...,...,...,...,...
4549,osjkh6,NaN,yellowmix,2021-07-27 09:32:14,NaN,This Is the End of Affirmative Action. What ar...,t5_2qhgd
4550,iw0q5sn,t3_yr5o90,yellowmix,2022-11-12 01:09:36,NaN,What do you mean? That nepotism comes from whi...,t5_2qhgd
4551,uyzxgz,NaN,Kamala_Metamorph,2022-05-27 14:52:16,​,How to have a conversation with an open-minded...,t5_38jid
4552,l20ftbs,t3_1cfw1ru,Extension_River_9901,2024-04-30 22:56:39,New user,Democrats that want to expand education .Fun...,t5_3amv4


Note: 307 rows were of deleted comments and so dropped from dataframe

### Drop rows that have been filtered by the AutoModerator

In [17]:
moderated_rows = comments_df[comments_df['username']=='AutoModerator']
print(f'Num of moderated rows: {moderated_rows.shape[0]}')

comments_df.drop(moderated_rows.index, inplace=True)
comments_df.reset_index(drop=True, inplace=True)
comments_df

Num of moderated rows: 0


,id,parent_id,username,time_created,flair,body,subreddit
0,14m8mf4,NaN,Tungsten_,2023-06-29 10:54:44,NaN,[Megathread] Supreme Court Ruling on Affirmati...,t5_2rfyw
1,jq5du0z,t3_14m8mf4,Tungsten_,2023-06-30 11:33:11,NaN,Thanks to everyone who engaged in insightful a...,t5_2rfyw
2,jq0dgzx,t3_14m8mf4,ProudBlackMatt,2023-06-29 11:16:15,Chinese-American,I would prefer using a process that takes into...,t5_2rfyw
3,jq0cg7k,t3_14m8mf4,TomatoCanned,2023-06-29 11:09:47,NaN,"u/Tungsten_, Thanks for creating a section jus...",t5_2rfyw
4,jq0f52k,t3_14m8mf4,bad-fengshui,2023-06-29 11:26:41,NaN,As with anything related to Asians in politics...,t5_2rfyw
...,...,...,...,...,...,...,...
4515,osjkh6,NaN,yellowmix,2021-07-27 09:32:14,NaN,This Is the End of Affirmative Action. What ar...,t5_2qhgd
4516,iw0q5sn,t3_yr5o90,yellowmix,2022-11-12 01:09:36,NaN,What do you mean? That nepotism comes from whi...,t5_2qhgd
4517,uyzxgz,NaN,Kamala_Metamorph,2022-05-27 14:52:16,​,How to have a conversation with an open-minded...,t5_38jid
4518,l20ftbs,t3_1cfw1ru,Extension_River_9901,2024-04-30 22:56:39,New user,Democrats that want to expand education .Fun...,t5_3amv4


### Tokenize

In [18]:
nlp = spacy.load('en_core_web_sm')

In [19]:
def tokenize_str(str):
    tokenized = []
    doc = nlp(str)
    for token in doc:
        if not token.is_punct and len(token.text.strip()) > 0:
            tokenized.append(token.text)
    return tokenized

In [20]:
# create new column of tokens
comments_df['tokens'] = comments_df['body'].apply(lambda x: tokenize_str(x))

In [21]:
comments_df.head(15)

,id,parent_id,username,time_created,flair,body,subreddit,tokens
0,14m8mf4,NaN,Tungsten_,2023-06-29 10:54:44,NaN,[Megathread] Supreme Court Ruling on Affirmati...,t5_2rfyw,"[Megathread, Supreme, Court, Ruling, on, Affir..."
1,jq5du0z,t3_14m8mf4,Tungsten_,2023-06-30 11:33:11,NaN,Thanks to everyone who engaged in insightful a...,t5_2rfyw,"[Thanks, to, everyone, who, engaged, in, insig..."
2,jq0dgzx,t3_14m8mf4,ProudBlackMatt,2023-06-29 11:16:15,Chinese-American,I would prefer using a process that takes into...,t5_2rfyw,"[I, would, prefer, using, a, process, that, ta..."
3,jq0cg7k,t3_14m8mf4,TomatoCanned,2023-06-29 11:09:47,NaN,"u/Tungsten_, Thanks for creating a section jus...",t5_2rfyw,"[u, Tungsten, Thanks, for, creating, a, sectio..."
4,jq0f52k,t3_14m8mf4,bad-fengshui,2023-06-29 11:26:41,NaN,As with anything related to Asians in politics...,t5_2rfyw,"[As, with, anything, related, to, Asians, in, ..."
5,jq0mhrx,t3_14m8mf4,Pancake_muncher,2023-06-29 12:12:39,NaN,Yet colleges will allow alumni and doners in e...,t5_2rfyw,"[Yet, colleges, will, allow, alumni, and, done..."
6,jq0mlbs,t3_14m8mf4,suberry,2023-06-29 12:13:16,NaN,I just hated Affirmative Action as a distracti...,t5_2rfyw,"[I, just, hated, Affirmative, Action, as, a, d..."
7,jq0cvvn,t3_14m8mf4,Puzzled-Painter3301,2023-06-29 11:12:31,NaN,My own feeling is that I was never in love wit...,t5_2rfyw,"[My, own, feeling, is, that, I, was, never, in..."
8,jq0jtzk,t3_14m8mf4,e9967780,2023-06-29 11:55:52,NaN,Anti Asian racism whether against East Asians ...,t5_2rfyw,"[Anti, Asian, racism, whether, against, East, ..."
9,jq0kfzq,t3_14m8mf4,NaN,2023-06-29 11:59:41,NaN,Can we overturn legacy and athlete admissions ...,t5_2rfyw,"[Can, we, overturn, legacy, and, athlete, admi..."


Problem with Reddit comments: We don't want to get split on (get rid of) forward slashes nor get rid of punctuations on usernames like "u/Tungsten_"

So, instead of using the native Spacy package, we use RedditScore which is built on Spacy but modified for Reddit/Twitter comments.

In [10]:
sample_comment = comments_df['body'][2]
print(sample_comment)

u/Tungsten_, Thanks for creating a section just to discuss this. When I read the news I immediately went searching for a forum where folks might have civil discourse on this topic.

Just had a few comments/questions:

1. Has anyone come across seemingly legitimate data sets on asians & college admission with respect to Affirmative Action (AA for short going forward)
2. As an Asian (not born in the US but pretty much assimilated here for 35+ years), I am conflicted. Research results like this one show: [https://www.pewresearch.org/race-ethnicity/2023/06/08/asian-americans-hold-mixed-views-around-affirmative-action/](https://www.pewresearch.org/race-ethnicity/2023/06/08/asian-americans-hold-mixed-views-around-affirmative-action/) that something like 53% Asians think AA is a good thing, and yet when you scroll down and look at the question of "Should colleges consider race/ethnicity in college admissions," the percentage of Asians that say yes are at 21%, no at 76%.

I am part of the 76%.

### Special case in Spacy 

'CrazyTokenizer' doesn't work bc the code is reliant on an old version of Spacy. So, we will create a special case in Spacy's tokenizer

In [24]:
from spacy.symbols import ORTH

doc = nlp(' Hi, u/Tungsten_, Thanks for creating a section')
print([w.text for w in doc])

special_case = [{'TEXT': {'REGEX': r'u\/([[:word:]]|-){3,23}'}}]
nlp.tokenizer.add_special_case('redditor', special_case)
doc = nlp(' Hi, u/Tungsten_, Thanks for creating a section')

print(print([w.text for w in doc]))

[' ', 'Hi', ',', 'u', '/', 'Tungsten', '_', ',', 'Thanks', 'for', 'creating', 'a', 'section']


KeyError: 65

In [8]:
import re
from spacy.tokenizer import Tokenizer

nlp.tokenizer = Tokenizer(nlp.vocab, token_match=re.compile(r'u/{1}\w{3,23}').search)

In [9]:
doc = nlp(sample_comment)

print([w.text for w in doc])

NameError: name 'sample_comment' is not defined

In [10]:
print(comments_df['tokens'][2])

['u', 'Tungsten', 'Thanks', 'for', 'creating', 'a', 'section', 'just', 'to', 'discuss', 'this', 'When', 'I', 'read', 'the', 'news', 'I', 'immediately', 'went', 'searching', 'for', 'a', 'forum', 'where', 'folks', 'might', 'have', 'civil', 'discourse', 'on', 'this', 'topic', 'Just', 'had', 'a', 'few', 'comments', 'questions', '1', 'Has', 'anyone', 'come', 'across', 'seemingly', 'legitimate', 'data', 'sets', 'on', 'asians', 'college', 'admission', 'with', 'respect', 'to', 'Affirmative', 'Action', 'AA', 'for', 'short', 'going', 'forward', '2', 'As', 'an', 'Asian', 'not', 'born', 'in', 'the', 'US', 'but', 'pretty', 'much', 'assimilated', 'here', 'for', '35', '+', 'years', 'I', 'am', 'conflicted', 'Research', 'results', 'like', 'this', 'one', 'show', 'https://www.pewresearch.org/race-ethnicity/2023/06/08/asian-americans-hold-mixed-views-around-affirmative-action/](https://www.pewresearch.org/race-ethnicity/2023/06/08/asian-americans-hold-mixed-views-around-affirmative-action/', 'that', 'some

We can see from above that the tokenization got worse when we added the token_match criteria. We need to modify the English class-attribute before loading model.

In [14]:
import spacy
nlp = spacy.load('en_core_web_sm')

#nlp.tokenizer.token_match = None (run line if results are same)
doc1 = nlp(sample_comment)
print([w.text for w in doc1])

# add token_match to tokenizer
nlp.tokenizer.token_match = re.compile(r'u/{1}\w{3,23}').match
doc2 = nlp(sample_comment)
print([w.text for w in doc2])

NameError: name 'sample_comment' is not defined

For some reason, the reddit username includes the comma.
*IDEA: We can strip the string of any characters that are not valid in a username.*

Our tokenization of reddit usernames is not perfect, it may include trailing punctuation. But let's move on.

In [70]:
#redditor_regex = re.compile(r'u/{1}\w{3,23}')
#print(redditor_regex.search(sample_comment))
#redditor_regex.match(sample_comment)

<re.Match object; span=(0, 11), match='u/Tungsten_'>


<re.Match object; span=(0, 11), match='u/Tungsten_'>

In [15]:
comments_df.head(5)

,username,flair_text,body,tokens
0,Tungsten_,NaN,Thanks to everyone who engaged in insightful a...,"[Thanks, to, everyone, who, engaged, in, insig..."
1,ProudBlackMatt,Chinese-American,I would prefer using a process that takes into...,"[I, would, prefer, using, a, process, that, ta..."
2,TomatoCanned,NaN,"u/Tungsten_, Thanks for creating a section jus...","[u, Tungsten, Thanks, for, creating, a, sectio..."
3,bad-fengshui,NaN,As with anything related to Asians in politics...,"[As, with, anything, related, to, Asians, in, ..."
4,Pancake_muncher,NaN,Yet colleges will allow alumni and doners in e...,"[Yet, colleges, will, allow, alumni, and, done..."


In [25]:
nlp = spacy.load('en_core_web_sm')
nlp.tokenizer.token_match = re.compile(r'u/{1}\w{3,23}').match # match reddit usernames

# drop old tokens column from data frame
comments_df.drop(['tokens'], axis=1, inplace=True)

In [26]:
# add column to data frame
comments_df['tokens_new'] = comments_df['body'].apply(lambda x: tokenize_str(x)) # takes a minute and half to run

In [27]:
comments_df.head(15)

,id,parent_id,username,time_created,flair,body,subreddit,tokens_new
0,14m8mf4,NaN,Tungsten_,2023-06-29 10:54:44,NaN,[Megathread] Supreme Court Ruling on Affirmati...,t5_2rfyw,"[Megathread, Supreme, Court, Ruling, on, Affir..."
1,jq5du0z,t3_14m8mf4,Tungsten_,2023-06-30 11:33:11,NaN,Thanks to everyone who engaged in insightful a...,t5_2rfyw,"[Thanks, to, everyone, who, engaged, in, insig..."
2,jq0dgzx,t3_14m8mf4,ProudBlackMatt,2023-06-29 11:16:15,Chinese-American,I would prefer using a process that takes into...,t5_2rfyw,"[I, would, prefer, using, a, process, that, ta..."
3,jq0cg7k,t3_14m8mf4,TomatoCanned,2023-06-29 11:09:47,NaN,"u/Tungsten_, Thanks for creating a section jus...",t5_2rfyw,"[u/Tungsten_,, Thanks, for, creating, a, secti..."
4,jq0f52k,t3_14m8mf4,bad-fengshui,2023-06-29 11:26:41,NaN,As with anything related to Asians in politics...,t5_2rfyw,"[As, with, anything, related, to, Asians, in, ..."
5,jq0mhrx,t3_14m8mf4,Pancake_muncher,2023-06-29 12:12:39,NaN,Yet colleges will allow alumni and doners in e...,t5_2rfyw,"[Yet, colleges, will, allow, alumni, and, done..."
6,jq0mlbs,t3_14m8mf4,suberry,2023-06-29 12:13:16,NaN,I just hated Affirmative Action as a distracti...,t5_2rfyw,"[I, just, hated, Affirmative, Action, as, a, d..."
7,jq0cvvn,t3_14m8mf4,Puzzled-Painter3301,2023-06-29 11:12:31,NaN,My own feeling is that I was never in love wit...,t5_2rfyw,"[My, own, feeling, is, that, I, was, never, in..."
8,jq0jtzk,t3_14m8mf4,e9967780,2023-06-29 11:55:52,NaN,Anti Asian racism whether against East Asians ...,t5_2rfyw,"[Anti, Asian, racism, whether, against, East, ..."
9,jq0kfzq,t3_14m8mf4,NaN,2023-06-29 11:59:41,NaN,Can we overturn legacy and athlete admissions ...,t5_2rfyw,"[Can, we, overturn, legacy, and, athlete, admi..."


### Filter and normalize text

1. Make all words lowercase (trivial)
2. Drop non-word tokens (may already be done in tokenize_str() function)
3. Remove stop-words (in a sophisticated manner)
4. Stem words to remove suffixes, prefixes, infixes OR Lemmatize tokens (intelligently)

#### 3. Remove stop-words

In [13]:
# Remove stop words using word counts
counts_dict = {}
for word in comments_df['tokens_new'].sum():
    word = word.lower()
    if word in counts_dict:
        counts_dict[word]+=1
    else:
        counts_dict[word] = 1

word_counts = sorted(counts_dict.items(), key = lambda x: x[1], reverse=True)
word_counts[:50]

[('the', 10504),
 ('to', 7373),
 ('of', 5903),
 ('and', 5694),
 ('a', 5293),
 ('that', 4814),
 ('in', 4436),
 ('i', 4103),
 ('is', 3984),
 ('it', 3495),
 ('are', 2626),
 ('you', 2537),
 ('for', 2504),
 ('asian', 2033),
 ('not', 2024),
 ("'s", 2019),
 ('this', 1859),
 ('as', 1843),
 ("n't", 1806),
 ('but', 1744),
 ('they', 1688),
 ('be', 1683),
 ('do', 1644),
 ('on', 1624),
 ('have', 1560),
 ('action', 1439),
 ('with', 1432),
 ('affirmative', 1415),
 ('we', 1334),
 ('asians', 1321),
 ('or', 1244),
 ('people', 1224),
 ('at', 1217),
 ('if', 1192),
 ('about', 1101),
 ('from', 1096),
 ('white', 1039),
 ('who', 1017),
 ('their', 981),
 ('more', 980),
 ('was', 959),
 ('there', 935),
 ('so', 900),
 ('an', 873),
 ('like', 846),
 ('just', 846),
 ('by', 844),
 ('what', 838),
 ('other', 837),
 ('because', 830)]

In [12]:
word_counts[:35]

[('the', 11882),
 ('to', 8178),
 ('of', 6683),
 ('and', 6456),
 ('a', 5916),
 ('that', 5361),
 ('in', 4994),
 ('is', 4546),
 ('i', 4546),
 ('it', 4017),
 ('are', 2945),
 ('you', 2838),
 ('for', 2794),
 ('not', 2295),
 ("'s", 2276),
 ('asian', 2240),
 ('as', 2076),
 ("n't", 2060),
 ('this', 2042),
 ('but', 1954),
 ('they', 1902),
 ('be', 1866),
 ('do', 1863),
 ('on', 1860),
 ('have', 1776),
 ('action', 1647),
 ('affirmative', 1630),
 ('with', 1589),
 ('we', 1467),
 ('asians', 1454),
 ('people', 1420),
 ('or', 1383),
 ('at', 1350),
 ('if', 1316),
 ('from', 1274)]

Mark/remove words as stop words that are more frequent than the first noun ('i').

In [14]:
stop_words_freq = []
for word, count in word_counts:
    if word == 'i':
        break
    else:
        stop_words_freq.append(word)

print(stop_words_freq)

['the', 'to', 'of', 'and', 'a', 'that', 'in']


Judging from the list, we can add 'is' and 'it' to list of stop words

In [15]:
stop_words_freq += ['is', 'it', "'m", '>', '<']
stop_words_freq

['the', 'to', 'of', 'and', 'a', 'that', 'in', 'is', 'it', "'m", '>', '<']

Function to normalize tokens:

In [28]:
def normalize_tokens(word_list, extra_stop_words=[]):
    normalized = []
    if type(word_list) == list and len(word_list) == 1:
        word_list = word_list[0]

    if type(word_list) == list:
        word_list = ' '.join([str(elem) for elem in word_list])

    doc = nlp(word_list.lower()) # lowercase words in word_list

    # add lexeme property of stopword to words considered as stopwords
    if len(extra_stop_words) > 0:
        for stopword in extra_stop_words:
            lexeme = nlp.vocab[stopword]
            lexeme.is_stop = True

    for w in doc:
        # if not stop word or punctuation, add it to normalized
        if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num and len(w.text.strip()) > 0:
            # add lemmatized version of word
            normalized.append(str(w.lemma_))

    return normalized

In [29]:
# add normalized tokens and count columns
comments_df['word_count'] = comments_df['tokens_new'].apply(lambda x: len(x))
comments_df['normalized_tokens'] = comments_df['tokens_new'].apply(lambda x: normalize_tokens(x))
comments_df['normalized_tokens_count'] = comments_df['normalized_tokens'].apply(lambda x: len(x))

,id,parent_id,username,time_created,flair,body,subreddit,tokens_new,word_count,normalized_tokens,normalized_tokens_count
0,14m8mf4,NaN,Tungsten_,2023-06-29 10:54:44,NaN,[Megathread] Supreme Court Ruling on Affirmati...,t5_2rfyw,"[Megathread, Supreme, Court, Ruling, on, Affir...",78,"[megathread, supreme, court, ruling, affirmati...",62
1,jq5du0z,t3_14m8mf4,Tungsten_,2023-06-30 11:33:11,NaN,Thanks to everyone who engaged in insightful a...,t5_2rfyw,"[Thanks, to, everyone, who, engaged, in, insig...",20,"[thank, engage, insightful, respectful, discou...",9
2,jq0dgzx,t3_14m8mf4,ProudBlackMatt,2023-06-29 11:16:15,Chinese-American,I would prefer using a process that takes into...,t5_2rfyw,"[I, would, prefer, using, a, process, that, ta...",103,"[prefer, process, take, account, poverty, inst...",52
3,jq0cg7k,t3_14m8mf4,TomatoCanned,2023-06-29 11:09:47,NaN,"u/Tungsten_, Thanks for creating a section jus...",t5_2rfyw,"[u/Tungsten_,, Thanks, for, creating, a, secti...",269,"[u/tungsten_,, thank, create, section, discuss...",126
4,jq0f52k,t3_14m8mf4,bad-fengshui,2023-06-29 11:26:41,NaN,As with anything related to Asians in politics...,t5_2rfyw,"[As, with, anything, related, to, Asians, in, ...",59,"[relate, asians, politic, m, see, lot, non, as...",25
5,jq0mhrx,t3_14m8mf4,Pancake_muncher,2023-06-29 12:12:39,NaN,Yet colleges will allow alumni and doners in e...,t5_2rfyw,"[Yet, colleges, will, allow, alumni, and, done...",40,"[college, allow, alumnus, doner, easily, consi...",19
6,jq0mlbs,t3_14m8mf4,suberry,2023-06-29 12:13:16,NaN,I just hated Affirmative Action as a distracti...,t5_2rfyw,"[I, just, hated, Affirmative, Action, as, a, d...",171,"[hate, affirmative, action, distraction, banda...",78
7,jq0cvvn,t3_14m8mf4,Puzzled-Painter3301,2023-06-29 11:12:31,NaN,My own feeling is that I was never in love wit...,t5_2rfyw,"[My, own, feeling, is, that, I, was, never, in...",231,"[feeling, love, affirmative, action, possible,...",102
8,jq0jtzk,t3_14m8mf4,e9967780,2023-06-29 11:55:52,NaN,Anti Asian racism whether against East Asians ...,t5_2rfyw,"[Anti, Asian, racism, whether, against, East, ...",46,"[anti, asian, racism, east, asians, south, asi...",21
9,jq0kfzq,t3_14m8mf4,NaN,2023-06-29 11:59:41,NaN,Can we overturn legacy and athlete admissions ...,t5_2rfyw,"[Can, we, overturn, legacy, and, athlete, admi...",29,"[overturn, legacy, athlete, admission, point, ...",15


In [31]:
comments_df

,id,parent_id,username,time_created,flair,body,subreddit,tokens_new,word_count,normalized_tokens,normalized_tokens_count
0,14m8mf4,NaN,Tungsten_,2023-06-29 10:54:44,NaN,[Megathread] Supreme Court Ruling on Affirmati...,t5_2rfyw,"[Megathread, Supreme, Court, Ruling, on, Affir...",78,"[megathread, supreme, court, ruling, affirmati...",62
1,jq5du0z,t3_14m8mf4,Tungsten_,2023-06-30 11:33:11,NaN,Thanks to everyone who engaged in insightful a...,t5_2rfyw,"[Thanks, to, everyone, who, engaged, in, insig...",20,"[thank, engage, insightful, respectful, discou...",9
2,jq0dgzx,t3_14m8mf4,ProudBlackMatt,2023-06-29 11:16:15,Chinese-American,I would prefer using a process that takes into...,t5_2rfyw,"[I, would, prefer, using, a, process, that, ta...",103,"[prefer, process, take, account, poverty, inst...",52
3,jq0cg7k,t3_14m8mf4,TomatoCanned,2023-06-29 11:09:47,NaN,"u/Tungsten_, Thanks for creating a section jus...",t5_2rfyw,"[u/Tungsten_,, Thanks, for, creating, a, secti...",269,"[u/tungsten_,, thank, create, section, discuss...",126
4,jq0f52k,t3_14m8mf4,bad-fengshui,2023-06-29 11:26:41,NaN,As with anything related to Asians in politics...,t5_2rfyw,"[As, with, anything, related, to, Asians, in, ...",59,"[relate, asians, politic, m, see, lot, non, as...",25
...,...,...,...,...,...,...,...,...,...,...,...
4515,osjkh6,NaN,yellowmix,2021-07-27 09:32:14,NaN,This Is the End of Affirmative Action. What ar...,t5_2qhgd,"[This, Is, the, End, of, Affirmative, Action, ...",15,"[end, affirmative, action, go]",4
4516,iw0q5sn,t3_yr5o90,yellowmix,2022-11-12 01:09:36,NaN,What do you mean? That nepotism comes from whi...,t5_2qhgd,"[What, do, you, mean, That, nepotism, comes, f...",32,"[mean, nepotism, come, white, supremacy, nepot...",15
4517,uyzxgz,NaN,Kamala_Metamorph,2022-05-27 14:52:16,​,How to have a conversation with an open-minded...,t5_38jid,"[How, to, have, a, conversation, with, an, ope...",237,"[conversation, open, minded, disadvantaged, gu...",117
4518,l20ftbs,t3_1cfw1ru,Extension_River_9901,2024-04-30 22:56:39,New user,Democrats that want to expand education .Fun...,t5_3amv4,"[Democrats, that, want, to, expand, education,...",349,"[democrats, want, expand, education, .funding,...",197


In [30]:
# check if normalized_tokens column is list or string - Result: list
print(type(comments_df['normalized_tokens'][0]))

<class 'list'>


Save above df as csv file.

In [32]:
comments_df.to_csv('../data/full_df_processed.csv')

In [23]:
input_file = 'comments.csv'
output_file = input_file[0:-4] + '_processed' + input_file[-4:]

print(output_file)

comments_processed.csv


### Exploratory Analysis

In [123]:
import nltk

In [124]:
comments_fq_dist = nltk.ConditionalFreqDist(((len(w), w) for w in comments_df['normalized_tokens'].sum()))

In [125]:
print(comments_fq_dist.N()) # number of total words

130955


In [135]:
comments_fq_dist[5].most_common(5) #most common 5-letter words

[('asian', 2442),
 ('white', 1321),
 ('think', 956),
 ('black', 787),
 ('group', 573)]